La idea principal que vamos a utilizar, es que no todos los caracteres de latin1 son caracteres válidos en español. Para ello, después de importar los módulos necesarios y las funciones que nos permiten pasar de caracteres en latin1 a números enteros y vice-versa, definimos como caracteres válidos solo aquellos que plausiblemente puedan ser utilizados en un texto en español.

In [1]:
import collections
import itertools
import random
import enchant
import binascii

def text_to_bits(text, encoding='latin1', errors='strict'):
    bits = bin(int.from_bytes(text.encode(encoding, errors), 'big'))[2:]
    return bits.zfill(8 * ((len(bits) + 7) // 8))

def text_from_bits(bits, encoding='latin1', errors='strict'):
    n = int(bits, 2)
    return n.to_bytes((n.bit_length() + 7) // 8, 'big').decode(encoding, errors) or '\0'

In [2]:
x = '0x0806170d1d1d4f029d0a1a440b081203309a074952081d0602110c440e014103201f1b1181004f0709450f111c040d12281a110b0601434309094907001f0e1d201f5424071c0a0f0504070b4f2f14162b17990452060e01810449000a4d1316261c0601131c4f021d100c08030c410724011000521c0a0e031108440a03410230165416074e1f0208170c440302411f29160296520f4f00030b06070a1f411629531c0c1702004d4c2808070003051c65160604520b0117030b0a011c4d141d24531509160b0e43080049120a040f0720531704010f1c43080049060e1f131c650a5406139f0e011e041f054f0e0e1d360706101b0a0e430d4505054f02131a291f1545160b4f1602451b89004d051665121310131d4f0705840f05010c125334061145010b4f131e000a0d1f04151227121a4502011d43190b49080a0e091c6517114502070a071e041a441f180d1a21120749520c0302020608174f1441162b1c0608171d4f0003080644071804052a005415000b070a1f119a16060e0e006b53310952031a0d080a49011d0c4107241d5417170d060602110c484f1c1416651e01061a0f1c430f0a1a051c4d02123716178813004f070945070b020f131669530d45020f1d024c080c0a0c040e1d24011804014e07020e8808441e180453361685041e0f1d0f0d164907000341162953100016014143260a1a8d4f2c131024171d0a522c1a0602018405434d100620531117134e0a0f4c0d06090d1f04532892074517031f11090b0d010b02135334061145010b4f15091784054f07001ea40054001c4e03024c0405000a0c4d532d121688134e0b0a1f151c011c190e532116541113024f0e03010644030c41032a001d061b9d0143080049080e1e4110240015165e4e1e1609450d011c090453311c1004014e1f0c088808440301041424010700520f03431e880644164d001124000000110b1d1009450d014f0c06062453170a1c4e0604190405440a1e070620010e0a5e4e16431817081e9c4d0d12365317041e020a104c06060a4f19001d651101001c4e1c0602110000004d100620531a0c1c091a0d0d450a051c0c410120101d079f0f4f0e8d164917000141023016540a061c0e430d4505054f050e01245310001e4e0c02000a1b4a4f280f53351c170a014e0e9203164544220c021c2b171b45141b0a43190b08440e01051624531984014e0011080007050b0c410a651f15071d1c060c1f0449151a0841103012181407070a110d450d014f01000065101b0b1d0d06070d16490c0e1e151265161a111d000c061f45190b1d4d120636530017171d0c0a090b1d0b1c4d0912271a00041c1a0a1042452c160e4d041d65051117160f0b43190b08440e010516245312001e07154f4c01060a0b08411d24171d00520b1d024c08081d001f411720530017170701170d450895001e410a65171b0b160b4f0d0d0100014f050011a8125408070b1d17034b0c26080e0909'
tamaño_llave = 32
msj = '0806170d1d1d4f029d0a1a440b081203309a074952081d0602110c440e014103201f1b1181004f0709450f111c040d12281a110b0601434309094907001f0e1d201f5424071c0a0f0504070b4f2f14162b17990452060e01810449000a4d1316261c0601131c4f021d100c08030c410724011000521c0a0e031108440a03410230165416074e1f0208170c440302411f29160296520f4f00030b06070a1f411629531c0c1702004d4c2808070003051c65160604520b0117030b0a011c4d141d24531509160b0e43080049120a040f0720531704010f1c43080049060e1f131c650a5406139f0e011e041f054f0e0e1d360706101b0a0e430d4505054f02131a291f1545160b4f1602451b89004d051665121310131d4f0705840f05010c125334061145010b4f131e000a0d1f04151227121a4502011d43190b49080a0e091c6517114502070a071e041a441f180d1a21120749520c0302020608174f1441162b1c0608171d4f0003080644071804052a005415000b070a1f119a16060e0e006b53310952031a0d080a49011d0c4107241d5417170d060602110c484f1c1416651e01061a0f1c430f0a1a051c4d02123716178813004f070945070b020f131669530d45020f1d024c080c0a0c040e1d24011804014e07020e8808441e180453361685041e0f1d0f0d164907000341162953100016014143260a1a8d4f2c131024171d0a522c1a0602018405434d100620531117134e0a0f4c0d06090d1f04532892074517031f11090b0d010b02135334061145010b4f15091784054f07001ea40054001c4e03024c0405000a0c4d532d121688134e0b0a1f151c011c190e532116541113024f0e03010644030c41032a001d061b9d0143080049080e1e4110240015165e4e1e1609450d011c090453311c1004014e1f0c088808440301041424010700520f03431e880644164d001124000000110b1d1009450d014f0c06062453170a1c4e0604190405440a1e070620010e0a5e4e16431817081e9c4d0d12365317041e020a104c06060a4f19001d651101001c4e1c0602110000004d100620531a0c1c091a0d0d450a051c0c410120101d079f0f4f0e8d164917000141023016540a061c0e430d4505054f050e01245310001e4e0c02000a1b4a4f280f53351c170a014e0e9203164544220c021c2b171b45141b0a43190b08440e01051624531984014e0011080007050b0c410a651f15071d1c060c1f0449151a0841103012181407070a110d450d014f01000065101b0b1d0d06070d16490c0e1e151265161a111d000c061f45190b1d4d120636530017171d0c0a090b1d0b1c4d0912271a00041c1a0a1042452c160e4d041d65051117160f0b43190b08440e010516245312001e07154f4c01060a0b08411d24171d00520b1d024c08081d001f411720530017170701170d450895001e410a65171b0b160b4f0d0d0100014f050011a8125408070b1d17034b0c26080e0909'

In [3]:
NUMEROS="0123456789"
CONSONANTES="BCDFGHJKLMNPQRSTVWXYZbcdfghjklmnpqrstvwxyzÑñ"
VOCALES="AEIOUaeiouÁÉÍÓÚáéíóúü"
LETRAS=CONSONANTES+VOCALES
SIMBOLOS=""" !"'(),.:;?@¡¸¿"""
LEGIBLES=NUMEROS+LETRAS+SIMBOLOS
print(LEGIBLES,len(LEGIBLES))
Lista = []
for letra in LEGIBLES:
    letra_en_latin1 = hex(int(text_to_bits(letra, encoding='latin1', errors='strict'),2))
    Lista.append(letra_en_latin1)
print(Lista)
for ele in Lista:
    binario = bin(int(ele,16))
    k = text_from_bits(binario, encoding='latin1', errors='strict')
    print(k)

0123456789BCDFGHJKLMNPQRSTVWXYZbcdfghjklmnpqrstvwxyzÑñAEIOUaeiouÁÉÍÓÚáéíóúü !"'(),.:;?@¡¸¿ 90
['0x30', '0x31', '0x32', '0x33', '0x34', '0x35', '0x36', '0x37', '0x38', '0x39', '0x42', '0x43', '0x44', '0x46', '0x47', '0x48', '0x4a', '0x4b', '0x4c', '0x4d', '0x4e', '0x50', '0x51', '0x52', '0x53', '0x54', '0x56', '0x57', '0x58', '0x59', '0x5a', '0x62', '0x63', '0x64', '0x66', '0x67', '0x68', '0x6a', '0x6b', '0x6c', '0x6d', '0x6e', '0x70', '0x71', '0x72', '0x73', '0x74', '0x76', '0x77', '0x78', '0x79', '0x7a', '0xd1', '0xf1', '0x41', '0x45', '0x49', '0x4f', '0x55', '0x61', '0x65', '0x69', '0x6f', '0x75', '0xc1', '0xc9', '0xcd', '0xd3', '0xda', '0xe1', '0xe9', '0xed', '0xf3', '0xfa', '0xfc', '0x20', '0x21', '0x22', '0x27', '0x28', '0x29', '0x2c', '0x2e', '0x3a', '0x3b', '0x3f', '0x40', '0xa1', '0xb8', '0xbf']
0
1
2
3
4
5
6
7
8
9
B
C
D
F
G
H
J
K
L
M
N
P
Q
R
S
T
V
W
X
Y
Z
b
c
d
f
g
h
j
k
l
m
n
p
q
r
s
t
v
w
x
y
z
Ñ
ñ
A
E
I
O
U
a
e
i
o
u
Á
É
Í
Ó
Ú
á
é
í
ó
ú
ü
 
!
"
'
(
)
,
.
:
;
?
@
¡
¸
¿


In [4]:
bloques_msj = int((len(x)-2)/tamaño_llave)

In [5]:
lista_var = [f't_{i}' for i in range(int(tamaño_llave/2))]
print(lista_var)

['t_0', 't_1', 't_2', 't_3', 't_4', 't_5', 't_6', 't_7', 't_8', 't_9', 't_10', 't_11', 't_12', 't_13', 't_14', 't_15']


Lo primero que vamos a hacer es crear un diccionario que nos guarde en la entrada $n$ la concatenación de los dígitos $2n$ y $2n+1$ para cada $n$ entre $0$ y $15$ para cada uno de los bloques, puesto que cada par de estos caracteres bajo la misma entrada en el diccionario estará encriptado con los mismos dos dígitos hexadecimales de la llave.

In [6]:
dic1 = {}
for i in range(16):
    lista_llave_i = []
    for j in range(0,len(msj),2):
        if (j % tamaño_llave) == 2*i:
            z = msj[j]+msj[j+1]
            lista_llave_i.append(z)
        dic1[f't_{i}'] = lista_llave_i
print(dic1)

{'t_0': ['08', '30', '20', '28', '20', '2b', '26', '24', '30', '29', '29', '65', '24', '20', '65', '36', '29', '65', '34', '27', '65', '21', '2b', '2a', '6b', '24', '65', '37', '69', '24', '36', '29', '24', '20', '28', '34', 'a4', '2d', '21', '2a', '24', '31', '24', '24', '24', '20', '36', '65', '20', '20', '30', '24', '35', '2b', '24', '65', '30', '65', '65', '36', '27', '65', '24', '24', '20', '65', 'a8'], 't_1': ['06', '9a', '1f', '1a', '1f', '17', '1c', '01', '16', '16', '53', '16', '53', '53', '0a', '07', '1f', '12', '06', '12', '17', '12', '1c', '00', '53', '1d', '1e', '16', '53', '01', '16', '53', '17', '53', '92', '06', '00', '12', '16', '00', '00', '1c', '01', '00', '53', '01', '53', '11', '53', '10', '16', '53', '1c', '17', '53', '1f', '12', '10', '16', '53', '1a', '05', '53', '17', '53', '17', '12'], 't_2': ['17', '07', '1b', '11', '54', '99', '06', '10', '54', '02', '1c', '06', '15', '17', '54', '06', '15', '13', '11', '1a', '11', '07', '06', '54', '31', '54', '01', '17', '

Ahora, chequeamos que cada par de dígitos hexadecimales de la llave en las posiciones $2n$ y $2n+1$ nos arroje caracteres válidos para todos los elementos asociados a la entrada $n$ del diccionario. Notemos que esto es computacionalmente muy rápido, ya que en total hay solo $16^2=256$ posíbilidades para los dós dígitos de la llave en la que estamos interesados. De esta forma, podemos hallar la llave de a dos dígitos a la vez, pudiendo saber si cada par de dígitos de la llave son potencialmente correctos con independencia de los demás, reducienco así el espacio de búsqueda de $(16^2)^{16}=16^{32}=2^{128}$ opciones, que es computacionalmente inviable, a tan solo $16\times 16^2=16^3=4096$ opciones, lo cuál es completamente factible.

In [7]:
La_Llave = {}
for dig in lista_var:
    listallave = []
    for i in range(16*16):
        found = True
        cont = 0
        for j in dic1[dig]:
            if found == False:
                #print('Error')
                cont = 0
                break
            k = bin(i ^ int(j,16))
            k_letra = text_from_bits(k, encoding='latin1', errors='strict')
            found = k_letra in LEGIBLES
            cont += 1
        if cont == 67:
            listallave.append(i)
    La_Llave[dig] = listallave
for k in La_Llave.values():
    print(k)

[69]
[115]
[116]
[101]
[114]
[110]
[111]
[99]
[108]
[101]
[105]
[100]
[111]
[109]
[97, 99]
[115]


In [8]:
print(La_Llave)

{'t_0': [69], 't_1': [115], 't_2': [116], 't_3': [101], 't_4': [114], 't_5': [110], 't_6': [111], 't_7': [99], 't_8': [108], 't_9': [101], 't_10': [105], 't_11': [100], 't_12': [111], 't_13': [109], 't_14': [97, 99], 't_15': [115]}


Finalmente, desencriptamos el texto

In [9]:
pt = ''
for j in range(0,len(msj),2):
    jmod16 = int(j/2)%16
    string_ct = msj[j]+msj[j+1]
    numero_ct = int(string_ct,16)
    ff = f't_{jmod16}'
    lista_ronda = La_Llave[ff]
    llave_ronda = lista_ronda[0]
    numero_pt = llave_ronda^numero_ct
    function_fodder = bin(numero_pt)
    car_pt = text_from_bits(function_fodder, encoding='latin1', errors='strict')
    pt += car_pt
print(pt)

Muchos años después, frente al pelotón de fusilamiento, el coronel Aureliano Buendía había de recordar aquella tarde remota en que su padre lo llevó a conocer el hielo. Macondo era entonces una aldea de veinte casas de barro y cañabrava construida a la orilla de un río de aguas diáfanas que se precipitaban por un lecho de piedras pulidas, blancas y enormes como huevos prehistóricos. El mundo era tan reciente, que muchas cosas carecían de nombre, y para mencionarlas había que señalarlas con el dedo. José Arcadio Buendía, que era el hombre más emprendedor que se vería jamás en la aldea, había dispuesto de tal modo la posición de las casas, que desde todas podía llegarse al río y abastecerse de agua con igual esfuerzo, y trazó las calles con tan buen sentido que ninguna casa recibía más sol que otra a la hora del calor. En pocos años, Macondo fue una aldea más ordenada y laboriosa que cualquiera de las conocidas hasta entonces por sus trescientos habitantes. Era en verdad una aldea feliz,